# Introduction
Below is a complete Python code for the Emotion Recognition project using MTCNN for face detection and HSEmotion pre-trained model for emotion classification. This code will process a video file, detect faces, classify emotions, and annotate the video with recognized emotions.

## Imports:

- `cv2` for OpenCV functions.
- `VideoFileClip` and `AudioFileClip` from the moviepy library to handle video and audio processing.
- `MTCNN` from the mtcnn library for face detection.
- `HSEmotionRecognizer` from the hsemotion library for emotion recognition.

## Face Detection:

The `detect_faces` function uses MTCNN to detect faces in a given frame.

## Emotion Classification:

The `classify_emotions` function uses HSEmotion to analyze the detected face image and classify emotions. The function returns the emotion with the highest confidence.

## Frame Annotation:

The `annotate_frame` function annotates each frame with rectangles around detected faces and labels them with the recognized emotions.

## Video Processing:

The `process_video` function reads the input video, processes each frame to detect and classify emotions, annotates the frames, and writes them to an output video file. It also merges the original audio with the processed video.

## Execution:

The script processes the input video and saves the annotated video to the specified output path. The processed video can then be downloaded using Google Colab's `files.download`.


In [ ]:
! pip install mtcnn hsemotion moviepy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 20.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvi

In [ ]:
from moviepy.editor import VideoFileClip, AudioFileClip
from mtcnn import MTCNN
from hsemotion.facial_emotions import HSEmotionRecognizer
import cv2

In [ ]:
def detect_faces(frame, detector):
    """ Detect faces in the frame """
    faces = detector.detect_faces(frame)
    return faces

In [ ]:
def classify_emotions(face_image, recognizer):
    """ Classify emotions for the given face image """
    results = recognizer.predict_emotions(face_image)
    if results:
        emotion = results[0]  # Get the most likely emotion
    else:
        emotion = 'Unknown'
    return emotion

In [ ]:
def annotate_frame(frame, faces, recognizer):
    """ Annotate the frame with recognized emotions """
    for face in faces:
        (x, y, w, h) = face['box']
        face_image = frame[y:y+h, x:x+w]  # Extract face region from frame
        emotion = classify_emotions(face_image, recognizer)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.putText(frame, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

In [ ]:
def process_video(video_path, output_path, slow_down_factor=1):
    detector = MTCNN()
    recognizer = HSEmotionRecognizer(model_name='enet_b0_8_best_vgaf', device='cpu')

    video_clip = VideoFileClip(video_path)
    fps = video_clip.fps

    temp_output_path = 'temp_output_video.mp4'
    temp_clip = video_clip.fl_image(lambda x: cv2.cvtColor(x, cv2.COLOR_RGB2BGR))
    temp_clip.write_videofile(temp_output_path, codec='libx264', audio_codec='aac')
    temp_clip.close()

    temp_clip = VideoFileClip(temp_output_path)

    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (int(temp_clip.size[0]), int(temp_clip.size[1])))

    for frame in temp_clip.iter_frames():
        faces = detect_faces(frame, detector)
        annotate_frame(frame, faces, recognizer)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        out.write(frame)

    out.release()
    temp_clip.close()
    cv2.destroyAllWindows()


In [ ]:
if __name__ == "__main__":
    video_path = '/content/جدو معتز.mp4'  # Update with your video path
    output_path = '/content/جدو معتز_output_video.mp4'  # Update with the desired output path
    slow_down_factor = 1  # Adjust the factor to slow down label changes
    process_video(video_path, output_path, slow_down_factor)



/root/.hsemotion/enet_b0_8_best_vgaf.pt Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)
Moviepy - Building video temp_output_video.mp4.
MoviePy - Writing audio in temp_output_videoTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video temp_output_video.mp4



t:  99%|█████████▉| 469/473 [00:12<00:00, 49.51it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /content/جدو معتز.mp4, 1082880 bytes wanted but 0 bytes read,at frame 470/473, at time 7.84/7.89 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+



Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 26ms/step


  warnings.warn("Warning: in file %s, "%(self.filename)+



1/1 [==============================] - 0s 25ms/step


  warnings.warn("Warning: in file %s, "%(self.filename)+



1/1 [==============================] - 0s 35ms/step


In [ ]:
from moviepy.editor import VideoFileClip, AudioFileClip
from mtcnn import MTCNN
from hsemotion.facial_emotions import HSEmotionRecognizer
import cv2

def detect_faces(frame, detector):
    """ Detect faces in the frame """
    faces = detector.detect_faces(frame)
    return faces

def classify_emotions(face_image, recognizer):
    """ Classify emotions for the given face image """
    results = recognizer.predict_emotions(face_image)
    if results:
        emotion = results[0]  # Get the most likely emotion
    else:
        emotion = 'Unknown'
    return emotion

def annotate_frame(frame, faces, recognizer):
    """ Annotate the frame with recognized emotions """
    for face in faces:
        (x, y, w, h) = face['box']
        face_image = frame[y:y+h, x:x+w]  # Extract face region from frame
        emotion = classify_emotions(face_image, recognizer)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.putText(frame, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

def process_video(video_path, output_path, slow_down_factor=1):
    detector = MTCNN()
    recognizer = HSEmotionRecognizer(model_name='enet_b0_8_best_vgaf', device='cpu')

    video_clip = VideoFileClip(video_path)
    fps = video_clip.fps

    # Get the audio from the original video
    audio_clip = video_clip.audio

    # Prepare the output video writer
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (int(video_clip.size[0]), int(video_clip.size[1])))

    # Process each frame in the video
    for frame in video_clip.iter_frames():
        # Detect faces and annotate emotions
        faces = detect_faces(frame, detector)
        annotate_frame(frame, faces, recognizer)

        # Convert frame to RGB (since moviepy returns frames in RGB)
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

        # Write the frame to the output video
        out.write(frame)

    # Release resources
    out.release()
    cv2.destroyAllWindows()

    # Combine the processed video with the original audio
    final_clip = VideoFileClip(output_path)
    final_clip = final_clip.set_audio(audio_clip)
    final_clip.write_videofile(output_path, codec='libx264', audio_codec='aac')

if __name__ == "__main__":
    video_path = '/content/سعيد صالح .mp4'  # Update with your video path
    output_path = '/content/سعيد صالح _output_video.mp4'  # Update with the desired output path
    slow_down_factor = 1  # Adjust the factor to slow down label changes
    process_video(video_path, output_path, slow_down_factor)


/root/.hsemotion/enet_b0_8_best_vgaf.pt Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)
1/1 [==============================] - 0s 224ms/step


1/1 [==============================] - 0s 30ms/step
Moviepy - Building video /content/سعيد صالح _output_video.mp4.
Moviepy - Writing video /content/سعيد صالح _output_video.mp4



t:   0%|          | 0/200 [00:00<?, ?it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /content/سعيد صالح _output_video.mp4, 90000 bytes wanted but 0 bytes read,at frame 13/201, at time 0.52/8.00 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in fi

Moviepy - Done !
Moviepy - video ready /content/سعيد صالح _output_video.mp4
